# Unit 12 - Tales from the Crypto

---


## 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [1]:
# Initial imports
import os
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import nltk as nltk
from newsapi import NewsApiClient
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Jordan\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
# Read your api key environment variable
load_dotenv()
api_key = os.getenv("news_api")

In [3]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=api_key)

In [4]:
# Fetch the Bitcoin news articles
btc_news = newsapi.get_everything(q="Bitcoin", from_param="2021-06-01")

In [5]:
# Fetch the Ethereum news articles
eth_news = newsapi.get_everything(q="Ethereum",from_param="2021-06-01")

In [6]:
def get_sent_df(topic_news):
    sentiments = []
    
    for article in topic_news["articles"]:
        try:
            text = article["content"]
            date = article["publishedAt"][:10]
            sentiment = analyzer.polarity_scores(text)
            compound = sentiment["compound"]
            pos = sentiment["pos"]
            neu = sentiment["neu"]
            neg = sentiment["neg"]
        
            sentiments.append({
                "text": text,
                "date": date,
                "compound": compound,
                "positive": pos,
                "negative": neg,
                "neutral": neu
            
            })

        except AttributeError:
            pass
    
    df = pd.DataFrame(sentiments)
    cols = ["date", "text", "compound", "positive", "negative", "neutral"]
    df = df[cols]
    
    return df

In [7]:
# Create the Bitcoin sentiment scores DataFrame
btc_sentiment = get_sent_df(btc_news)
btc_sentiment.head()

,date,text,compound,positive,negative,neutral
0,2021-06-05,"Hello friends, and welcome back to Week in Rev...",0.75,0.154,0.0,0.846
1,2021-06-08,"By Mary-Ann RussonBusiness reporter, BBC News\...",0.34,0.070,0.0,0.930
2,2021-06-08,Representations of the virtual currency Bitcoi...,0.00,0.000,0.0,1.000
3,2021-06-03,"Cryptocurrencies such as bitcoin are a ""farce""...",0.00,0.000,0.0,1.000
4,2021-06-05,"In a video broadcast to Bitcoin 2021, a multid...",0.00,0.000,0.0,1.000


In [8]:
# Create the Ethereum sentiment scores DataFrame
eth_sentiment = get_sent_df(eth_news)

In [9]:
# Describe the Bitcoin Sentiment
btc_sentiment.describe()

,compound,positive,negative,neutral
count,20.000000,20.000000,20.000000,20.000000
mean,0.223575,0.066400,0.020800,0.912850
std,0.301205,0.070866,0.037955,0.090939
min,-0.273200,0.000000,0.000000,0.728000
25%,0.000000,0.000000,0.000000,0.842250
50%,0.077150,0.056000,0.000000,0.930000
75%,0.475775,0.124000,0.016250,1.000000
max,0.750000,0.180000,0.113000,1.000000


In [10]:
# Describe the Ethereum Sentiment
eth_sentiment.describe()

,compound,positive,negative,neutral
count,20.000000,20.000000,20.000000,20.000000
mean,0.151885,0.061100,0.023800,0.915100
std,0.337584,0.075627,0.043086,0.096037
min,-0.458800,0.000000,0.000000,0.655000
25%,0.000000,0.000000,0.000000,0.863500
50%,0.000000,0.020500,0.000000,0.929500
75%,0.416125,0.103000,0.036750,1.000000
max,0.690800,0.246000,0.151000,1.000000


### Questions:

Q: Which coin had the highest mean positive score?

A: btc had a higher mean positive score.

Q: Which coin had the highest compound score?

A: btc had a higher compound score.

Q. Which coin had the highest positive score?

A: btc had a higher positive score.

---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [11]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [12]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [13]:
# Instantiate the lemmatizer
lemmatizer = WordNetLemmatizer()
# Create a list of stopwords
sw = set(stopwords.words('english'))

# Expand the default stopwords list if necessary
# YOUR CODE HERE!

In [14]:
# Complete the tokenizer function
def tokenizer(text):
    """Tokenizes text."""
    
    # Create a tokenized list of the words
    words = word_tokenize(text)
   
    # Convert the words to lowercase
    
    words = list(filter(lambda word: word.lower(), words))
   # lc_words = [word.lower() for word in words if word.lower() not in sw]
    
    # Remove the punctuation from text
    words = list(filter(lambda p: p not in punctuation, words))
    
    # Remove the stop words
    words = list(filter(lambda word: word not in sw, words))
    
    # Lemmatize words into root words
    tokens = list(filter(lambda word: lemmatizer.lemmatize(word), words))
    #tokens = [lemmatizer.lemmatize(word) for word in words]
    return tokens

In [15]:
# Create a new tokens column for Bitcoin
btc_sentiment["tokens"] = btc_sentiment["text"].apply(tokenizer)
btc_sentiment.head()

,date,text,compound,positive,negative,neutral,tokens
0,2021-06-05,"Hello friends, and welcome back to Week in Rev...",0.75,0.154,0.0,0.846,"[Hello, friends, welcome, back, Week, Review, ..."
1,2021-06-08,"By Mary-Ann RussonBusiness reporter, BBC News\...",0.34,0.070,0.0,0.930,"[By, Mary-Ann, RussonBusiness, reporter, BBC, ..."
2,2021-06-08,Representations of the virtual currency Bitcoi...,0.00,0.000,0.0,1.000,"[Representations, virtual, currency, Bitcoin, ..."
3,2021-06-03,"Cryptocurrencies such as bitcoin are a ""farce""...",0.00,0.000,0.0,1.000,"[Cryptocurrencies, bitcoin, ``, farce, '', sym..."
4,2021-06-05,"In a video broadcast to Bitcoin 2021, a multid...",0.00,0.000,0.0,1.000,"[In, video, broadcast, Bitcoin, 2021, multiday..."


In [16]:
# Create a new tokens column for Ethereum
eth_sentiment["tokens"] = eth_sentiment["text"].apply(tokenizer)
eth_sentiment.head()

,date,text,compound,positive,negative,neutral,tokens
0,2021-06-03,This new mining feature is called 'Norton Cryp...,0.0000,0.000,0.000,1.000,"[This, new, mining, feature, called, 'Norton, ..."
1,2021-06-04,CryptoPunks represent the only historically re...,-0.4588,0.067,0.151,0.782,"[CryptoPunks, represent, historically, relevan..."
2,2021-06-01,"Yao Qian, the former head of China's digital y...",0.3612,0.070,0.000,0.930,"[Yao, Qian, former, head, China, 's, digital, ..."
3,2021-06-01,A consumer uses a digital yuan red envelope in...,0.0000,0.000,0.000,1.000,"[A, consumer, uses, digital, yuan, red, envelo..."
4,2021-06-08,An update to an existing art app that allows a...,0.0000,0.000,0.000,1.000,"[An, update, existing, art, app, allows, artis..."


---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [17]:
from collections import Counter
from nltk import ngrams

In [18]:
def combine_texts(tokens_df):
    result = []
    for i in tokens_df:
        result.extend(i)
    return result

In [19]:
# def gen_bigrams(tokens_df): 
#     bigrams = tokens_df.apply(lambda x: ngrams(x, n=2))
#     result = []
#     for i in bigrams:
#         result.extend(i)
#     return result

In [20]:
def gen_bigrams(tokens_df): 
    bigrams = tokens_df.apply(lambda x: ngrams(x, n=2))
    bigrams = combine_texts(bigrams)
    return bigrams

In [21]:
# Generate the Bitcoin N-grams where N=2
btc_tokens = btc_sentiment["tokens"].copy()
btc_bigrams = gen_bigrams(btc_tokens)
print(btc_bigrams)

[('Hello', 'friends'), ('friends', 'welcome'), ('welcome', 'back'), ('back', 'Week'), ('Week', 'Review'), ('Review', 'Last'), ('Last', 'week'), ('week', 'I'), ('I', 'wrote'), ('wrote', 'tech'), ('tech', 'taking'), ('taking', 'Disney'), ('Disney', 'This'), ('This', 'week'), ('week', 'I'), ('I', '’'), ('’', 'talking'), ('talking', 'search'), ('search', 'new'), ('new', 'crypto'), ('crypto', 'messiah'), ('messiah', 'If'), ('If', 'youre'), ('youre', 'reading'), ('reading', 'Tec…'), ('Tec…', '+7741'), ('+7741', 'chars'), ('By', 'Mary-Ann'), ('Mary-Ann', 'RussonBusiness'), ('RussonBusiness', 'reporter'), ('reporter', 'BBC'), ('BBC', 'News'), ('News', 'image'), ('image', 'copyrightGetty'), ('copyrightGetty', 'Images'), ('Images', 'Former'), ('Former', 'US'), ('US', 'President'), ('President', 'Donald'), ('Donald', 'Trump'), ('Trump', 'told'), ('told', 'Fox'), ('Fox', 'Business'), ('Business', 'sees'), ('sees', 'Bitcoin'), ('Bitcoin', '``'), ('``', 'scam'), ('scam', "''"), ("''", 'affecting'), 

In [22]:
# Generate the Ethereum N-grams where N=2
eth_tokens = eth_sentiment["tokens"].copy()
eth_bigrams = gen_bigrams(eth_tokens)
print(eth_bigrams)

[('This', 'new'), ('new', 'mining'), ('mining', 'feature'), ('feature', 'called'), ('called', "'Norton"), ("'Norton", 'Crypto'), ('Crypto', 'rolling'), ('rolling', 'tomorrow'), ('tomorrow', 'Norton'), ('Norton', '360'), ('360', 'users'), ('users', 'enrolled'), ('enrolled', 'Norton'), ('Norton', "'s"), ("'s", 'early'), ('early', 'adopter'), ('adopter', 'program'), ('program', 'When'), ('When', 'Norton'), ('Norton', 'Crypto'), ('Crypto', 'enabled'), ('enabled', 'software'), ('software', 'use…'), ('use…', '+631'), ('+631', 'chars'), ('CryptoPunks', 'represent'), ('represent', 'historically'), ('historically', 'relevant'), ('relevant', 'narrative'), ('narrative', 'upon'), ('upon', 'value'), ('value', 'could'), ('could', 'conceivably'), ('conceivably', 'live'), ('live', 'decades'), ('decades', 'within'), ('within', 'expanding'), ('expanding', 'catalogue'), ('catalogue', 'raisonné'), ('raisonné', 'Ill'), ('Ill', 'damned'), ('damned', 'Im'), ('Im', 'counting'), ('counting', 'Jacks'), ('Jacks'

In [23]:
# Function token_count generates the top 10 words for a given coin
def token_count(tokens, N=10):
    """Returns the top N tokens from the frequency count"""
    return dict(Counter(tokens).most_common(N))

In [28]:
# Use token_count to get the top 10 words for Bitcoin
btc_ten = token_count(combine_texts(btc_tokens))
btc_ten_df = pd.DataFrame(btc_ten,columns=["word", "count"])
btc_ten_df

,word,count
0,chars,20
1,Bitcoin,14
2,bitcoin,7
3,cryptocurrency,7
4,illustration,6
5,2021,6
6,picture,5
7,taken,5
8,seen,5
9,President,4


In [29]:
# Use token_count to get the top 10 words for Ethereum
eth_ten = token_count(combine_texts(eth_tokens))
eth_ten_df = pd.DataFrame(eth_ten,columns=["word", "count"])
eth_ten_df

,word,count
0,chars,20
1,digital,6
2,Ethereum,6
3,Bitcoin,6
4,market,5
5,cryptocurrencies,5
6,'s,4
7,crypto,4
8,seen,4
9,illustration,4


---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [32]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [36]:
# Generate the Bitcoin word cloud
comb_df = combine_texts(btc_tokens)
# btc_wc = WordCloud().generate()
# plt.imshow(btc_wc)
print(comb_df)

['Hello', 'friends', 'welcome', 'back', 'Week', 'Review', 'Last', 'week', 'I', 'wrote', 'tech', 'taking', 'Disney', 'This', 'week', 'I', '’', 'talking', 'search', 'new', 'crypto', 'messiah', 'If', 'youre', 'reading', 'Tec…', '+7741', 'chars', 'By', 'Mary-Ann', 'RussonBusiness', 'reporter', 'BBC', 'News', 'image', 'copyrightGetty', 'Images', 'Former', 'US', 'President', 'Donald', 'Trump', 'told', 'Fox', 'Business', 'sees', 'Bitcoin', '``', 'scam', "''", 'affecting', 'value', 'US', 'dolla…', '+3638', 'chars', 'Representations', 'virtual', 'currency', 'Bitcoin', 'stand', 'motherboard', 'picture', 'illustration', 'taken', 'May', '20', '2021', 'REUTERS/Dado', 'Ruvic/Illustration/File', 'PhotoMicroStrategy', 'Inc', 'MSTR.O', 'major', 'b…', '+954', 'chars', 'Cryptocurrencies', 'bitcoin', '``', 'farce', "''", 'symptom', 'bubbles', 'forming', 'financial', 'markets', 'Amundi', 'chief', 'investment', 'officer', 'Pascal', 'Blanque', 'said', 'Thursday', 'Bitcoin', 'trading', 'around', '39', '…', '+

In [23]:
# Generate the Ethereum word cloud
# YOUR CODE HERE!

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [24]:
import spacy
from spacy import displacy

In [25]:
# Download the language model for SpaCy
# !python -m spacy download en_core_web_sm

In [26]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

---
### Bitcoin NER

In [27]:
# Concatenate all of the Bitcoin text together
# YOUR CODE HERE!

In [28]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [29]:
# Render the visualization
# YOUR CODE HERE!

In [30]:
# List all Entities
# YOUR CODE HERE!

---

### Ethereum NER

In [31]:
# Concatenate all of the Ethereum text together
# YOUR CODE HERE!

In [32]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [33]:
# Render the visualization
# YOUR CODE HERE!

In [34]:
# List all Entities
# YOUR CODE HERE!

---